In [1]:
import os
import shutil
import pandas as pd


In [2]:
# function to check number of images in folders
import os

def count_images_in_directory(directory):
    """
    Count the number of image files in a directory.
    """
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
    image_count = 0

    # Iterate through all files in the directory
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        # Check if the file is a regular file (not a directory)
        if os.path.isfile(file_path):
            # Check if the file has a recognized image extension
            if any(file_name.lower().endswith(ext) for ext in image_extensions):
                image_count += 1

    return image_count

def count_images_in_all_directories(root_directory):
    """
    Count the number of image files in all directories and subdirectories.
    """
    total_image_count = 0

    # Walk through all directories and subdirectories
    for dirpath, dirnames, filenames in os.walk(root_directory):
        # Count the number of image files in the current directory
        current_dir_image_count = count_images_in_directory(dirpath)

        # Add the count to the total image count
        total_image_count += current_dir_image_count

        # Print the directory path and the number of images
        #print(f"Directory: {dirpath}, Images: {current_dir_image_count}")

    return total_image_count


### ADULTS: Only move files from RFW full also in balanced adulst csv folder
**OBS: to file structure as child data**

In [3]:
#adults_csv = pd.read_csv('../data/adults_balanced.csv')
adults_csv = pd.read_csv('/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/adults_balanced.csv')

len(adults_csv)

3306

In [5]:
adults_csv

,img_name,im_id,ethnicity,id_class,face_id,Age,children_agegroup
0,m.0fw3k1_0002,m.0fw3k1,African,0fw3k1,2,20,NaN
1,m.04zvg12_0002,m.04zvg12,African,04zvg12,2,24,NaN
2,m.05mrf5k_0002,m.05mrf5k,African,05mrf5k,2,28,NaN
3,m.07b34k_0003,m.07b34k,African,07b34k,3,44,NaN
4,m.02xgm5_0002,m.02xgm5,African,02xgm5,2,34,NaN
...,...,...,...,...,...,...,...
3301,m.0h949qw_0001,m.0h949qw,Indian,0h949qw,1,22,NaN
3302,m.0bhcm0z_0003,m.0bhcm0z,Indian,0bhcm0z,3,32,NaN
3303,m.0bgrs7_0002,m.0bgrs7,Indian,0bgrs7,2,20,NaN
3304,m.0b4y2t_0003,m.0b4y2t,Indian,0b4y2t,3,38,NaN


In [6]:
def move_adults(image_dir, dst_root):
    image_file_names = adults_csv['img_name'].tolist()

    # Get the list of folders in the image directory
    folders = [os.path.join(image_dir, d) for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
    # Iterate through each folder
    for folder in folders:
        # Get the list of subfolders
        folders2 = [os.path.join(folder, d) for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
        for folder2 in folders2:
        # Get the list of image files in the folder
            files = [f for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))]

            # Check if any of the image file names from the CSV are in the folder
            for file in files:
                if file[:-4] in image_file_names:
                    # Move the image file to the output directory

                    # Make folder with ethnicityname_imageid
                    folder2_add = folder2.split("/")[-2:]
                    folder2_add_real = '_'.join(folder2_add)

                    dst_root = dst_root
                    src = os.path.join(folder2, file)
                    dst = os.path.join(dst_root, folder2_add_real)
                    os.makedirs(dst,exist_ok=True)
                    shutil.copy(src, dst)
    print('Done moving files')

In [8]:

image_dir = '../data/raw_full/RFW/data'
dst_root = '../data/raw_full/adults' #../data/raw_full/adults'

### Only run if you need to move data

#move_adults(image_dir, dst_root)

Done moving files


In [9]:
## Check number of images in folder
root_directory = '/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults'
total_images = count_images_in_all_directories(root_directory)
print(f"Total number of images: {total_images}")

Total number of images: 3307


In [11]:
# Where's the duplicates?

# Function to find duplicate file names within a folder and remove the first instance
def remove_first_instance_of_duplicate_file_names(folder_path):
    file_names = {}  # Dictionary to store file names and their counts

    # Traverse each folder recursively
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # Increment count for file name in dictionary
            # if file_name in file_names:
            #     # If the file name already exists, delete the first occurrence
            #     os.remove(file_path)
            #     print(f"Removed first instance of duplicate file: {file_path}")
            # else:
            #     file_names[file_name] = file_path

            if file_name in file_names:
                # If the file name already exists, remove the entire containing folder
                os.system(f"rm -rf '{os.path.dirname(file_path)}'")
                print(f"Removed folder containing duplicate file: {os.path.dirname(file_path)}")
                break  # Once a duplicate is found and removed, exit the loop
            else:
                file_names[file_name] = file_path

# Main function to check for duplicate file names in all folders
def check_for_duplicate_file_names_and_remove_first_instance(root_folder):
    for root, dirs, files in os.walk(root_folder):
        print(f"Checking duplicates in folder: {root}")
        remove_first_instance_of_duplicate_file_names(root)

# Example usage:
root_folder = '/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults'
check_for_duplicate_file_names_and_remove_first_instance(root_folder)


Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults
Removed folder containing duplicate file: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults/Indian_m.0gc2xf9
Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults/Asian_m.02w_xk
Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults/Asian_m.03d5c56
Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults/African_m.0dr_s5w
Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults/Caucasian_m.03qg_h5
Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults/Caucasian_m.05zx1f
Checking duplicates in folder: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/d

In [12]:
## Check number of images in folder now
root_directory = '/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/adults'
total_images = count_images_in_all_directories(root_directory)
print(f"Total number of images: {total_images}") # should be 3306

Total number of images: 3306


### CHILDREN: Only move files from YLFW full and RFW full also in balanced children csv folder
**OBS: RFW to file structure as child data**

In [8]:
children_csv = pd.read_csv('/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/child_balanced.csv')
len(children_csv)

3306

In [6]:
def move_children(image_dir_rfw, image_dir_ylfw, dst_root, children_df):

    # First for RFW
    #rfw_csv = children_csv[children_csv.img_org_name.notna()]
    #image_file_names = rfw_csv['img_org_name'].tolist()
    rfw_csv = children_df[children_df["img_name"].str.contains('\.')]
    image_file_names = rfw_csv['img_name'].tolist()

    # Get the list of folders in the image directory
    folders = [os.path.join(image_dir_rfw, d) for d in os.listdir(image_dir_rfw) if os.path.isdir(os.path.join(image_dir_rfw, d))]
    # Iterate through each folder
    for folder in folders:
        # Get the list of subfolders
        folders2 = [os.path.join(folder, d) for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
        for folder2 in folders2:
        # Get the list of image files in the folder
            files = [f for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))]

            # Check if any of the image file names from the CSV are in the folder
            for file in files:
                if file[:-4] in image_file_names:
                    # Move the image file to the output directory

                    # Make folder with ethnicityname_imageid
                    folder2_add = folder2.split("/")[-2:]
                    folder2_add_real = '_'.join(folder2_add)

                    dst_root = dst_root
                    src = os.path.join(folder2, file)
                    dst = os.path.join(dst_root, folder2_add_real)
                    os.makedirs(dst,exist_ok=True)
                    shutil.copy(src, dst)
    print('Done moving RFW files')


    # THEN YLFW
    #ylfw_csv = children_csv[children_csv.img_org_name.isna()]
    ylfw_csv = children_df[~children_df["img_name"].str.contains('\.')]
    ylfw_image_file_names = ylfw_csv['img_name'].tolist()

    # Get the list of folders in the image directory
    folders = [os.path.join(image_dir_ylfw, d) for d in os.listdir(image_dir_ylfw) if os.path.isdir(os.path.join(image_dir_ylfw, d))]
    # Iterate through each folder
    for folder in folders:
        # Get the list of image files in the folder
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        # Check if any of the image file names from the CSV are in the folder
        for file in files:
            if file[:-4] in ylfw_image_file_names:
                # Move the image file to same foldername in the output directory
                folder_dst = folder.split("/")[-1]

                dst_root = dst_root
                src = os.path.join(folder, file)
                dst = os.path.join(dst_root, folder_dst)
                os.makedirs(dst,exist_ok=True)
                shutil.copy(src, dst)

    print('Done moving YLFW files')


In [9]:
image_dir_rfw = "../data/raw_full/RFW/data"
image_dir_ylfw = "../data/raw_full/YLFW/data_aligned"
dst_root = "../data/raw_full/children_test"

# Only run if you want to move data

move_children(image_dir_rfw, image_dir_ylfw, dst_root,children_csv)

KeyboardInterrupt: 

In [17]:
## Check number of images in folder
root_directory = '../data/raw_full/children'
total_images = count_images_in_all_directories(root_directory)
print(f"Total number of images: {total_images}")

Total number of images: 3306
